<a href="https://colab.research.google.com/github/taitip-supha/BADS7604_Group3/blob/main/experiment/6310422009_3layer_1-28node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (1 - RUN) Seting Import package and Parameter

ref : 

In [ ]:
#Set Environment and Parameter
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score ,f1_score ,precision_score ,recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb

import time

pd.set_option('display.max_columns', 100)



RANDOM_STATE = 2018
PREDS_RATIO = 0.5

# (2 - RUN) Prepar Data

## Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/BADS7604_DL/data/creditcard.csv")
data_df.head(3)
print(f"Size od data frame {data_df.shape}")

Size od data frame (284807, 31)


In [ ]:
#Define predictors and target values
target = 'Class'
predictors = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10','V11', 'V12', 'V13', 'V14', 'V15',
                   'V16', 'V17', 'V18', 'V19','V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

In [ ]:
df_x = data_df[predictors]
df_y = data_df[target]

## Scaling predictors

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_x = pd.DataFrame(scaler.fit_transform(df_x))

## Split data in train, test

In [ ]:
TEST_SIZE = 0.20 # test size using_train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=True )
print(f"Proportion of Target: {y_train.mean()}")

Proportion of Target: 0.0017204678619236761


## Oversampling Trainig Set

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = RANDOM_STATE ,sampling_strategy='minority')
x_train, y_train = smote.fit_resample(x_train,y_train)
print(f"Proportion after oversampling : {y_train.mean()}")

Proportion after oversampling : 0.5


## Prepare Data in Numpy

In [ ]:
#Predictor
x_train = x_train.values.astype(np.float32)
x_test  = x_test.values.astype(np.float32)
print( f" x_train.shape={x_train.shape}, x_train.dtype={x_train.dtype}, min(x_train)={np.min(x_train):0.4f}, max(x_train)={np.max(x_train):0.4f}")
print( f" x_test.shape={x_test.shape}, x_test.dtype={x_test.dtype}, min(x_test)={np.min(x_test):0.4f}, max(x_test)={np.max(x_test):0.4f}")

 x_train.shape=(454906, 29), x_train.dtype=float32, min(x_train)=0.0000, max(x_train)=1.0000
 x_test.shape=(56962, 29), x_test.dtype=float32, min(x_test)=0.0000, max(x_test)=1.0000


In [ ]:
#Target
y_train = y_train.values
y_test = y_test.values
print( f" y_train.shape={y_train.shape}, y_train.dtype={y_train.dtype}, min(y_train)={np.min(y_train)}, max(y_train)={np.max(y_train)}")
print( f" y_test.shape={y_test.shape}, y_test.dtype={y_test.dtype}, min(y_test)={np.min(y_test)}, max(y_test)={np.max(y_test)}")

 y_train.shape=(454906,), y_train.dtype=int64, min(y_train)=0, max(y_train)=1
 y_test.shape=(56962,), y_test.dtype=int64, min(y_test)=0, max(y_test)=1


# (3 - Run) Loop Training DP-MLP

## Set seed

In [ ]:
import sys
import tensorflow as tf
#seting
np.random.seed(1234)
tf.random.set_seed(5678)
_,input_dim = x_train.shape #the number of features per one input
output_din = len(set(y_train)) #the number of output classes

## Create Function and setting model

In [ ]:
def mlp_training(batchSize ,epochNum ,learnRate ,optzer ,nodes_num=1 ,nodes_act='relu'):
  model = tf.keras.models.Sequential()
  #Input layer
  model.add(tf.keras.Input(shape=(input_dim,)))
  #Hidden -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
  #> 1 layer
  model.add(tf.keras.layers.Dense(nodes_num, activation=nodes_act, name='hidden1')) 
  model.add(tf.keras.layers.BatchNormalization(axis =-1, name='bn1'))
  #> 2 layer
  model.add(tf.keras.layers.Dense(nodes_num, activation=nodes_act, name='hidden2')) 
  model.add(tf.keras.layers.BatchNormalization(axis =-1, name='bn2'))  
  #> 3 layer
  model.add(tf.keras.layers.Dense(nodes_num, activation=nodes_act, name='hidden3')) 
  model.add(tf.keras.layers.BatchNormalization(axis =-1, name='bn3'))  
  #----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
  #output layer
  model.add(tf.keras.layers.Dense(2, activation='softmax' , name='output'))

  #Compiling the model
  if optzer.lower() == 'adum':
    opt = tf.keras.optimizers.Adam(learning_rate=learnRate)
  elif optzer.lower() == 'sgd':
    opt = tf.keras.optimizers.SGD(learning_rate=learnRate)
  else :
    print('error optimizers')
  model.compile(optimizer=opt
                      , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
                      , metrics=['acc'])
  #Training
  history = model.fit(x_train, y_train, batch_size=batchSize, epochs=epochNum, verbose=0, validation_split=0.2)
  return model,history

## Loop

In [ ]:
#list of Parameter batchSize ,epochNum ,learnRate ,optzer ,nodes_num=1 ,nodes_act='relu'
lst_batch = [256, 512, 1024]
lst_epoch = [10, 25, 50]
lst_learn = [0.001 ,0.005]
lst_opt = ['adum' ,'sgd']
lst_node = [1,28,56]
lst_activation = ['sigmoid', 'relu']

In [ ]:
#Import last result
#For New Loop
#df_result_MLP = pd.DataFrame()
#Continue from last state
df_result_MLP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BADS7604_DL/data/mlp_result_l007_c009.csv')
last_step = len(df_result_MLP)
print(f"Last step is : {last_step}")
df_result_MLP.tail(2)

Last step is : 9


,batch_size,epoch,learning_rate,optimizer,num_node,activation,layers,runing_times,auc,f1score,precision,recall,loss,acc
7,256,10,0.001,sgd,1,relu,hidden1|bn1|hidden2|bn2|hidden3|bn3|output,82.772641,0.957397,0.991174,0.998181,0.985359,0.083462,0.985359
8,256,10,0.001,sgd,28,sigmoid,hidden1|bn1|hidden2|bn2|hidden3|bn3|output,82.738010,0.974822,0.992894,0.998264,0.988606,0.067524,0.988606


In [ ]:
count_loop = 0
start_tm = time.time()
for batch_i in lst_batch:
  for epoch_j in lst_epoch:
    for learn_k in lst_learn:
      for opt_l in lst_opt:
        for node_m in lst_node:
          for act_n in lst_activation:
            count_loop =count_loop+1
            if count_loop>last_step:
              inloop_tm = time.time()
              model,_ = mlp_training(batch_i,epoch_j,learn_k,opt_l,node_m,act_n)
              inloop_tm = time.time() - inloop_tm
              preds_mlp = model.predict(x_test)
              results = model.evaluate(x_test, y_test, batch_size=128 , verbose=0)
              #Save Result
              result_dic = {'batch_size':[batch_i]
                              ,'epoch':[epoch_j]
                              ,'learning_rate':[learn_k]
                              ,'optimizer':[opt_l]
                              ,'num_node':[node_m]
                              ,'activation':[act_n]
                              ,'layers':['|'.join([layer.name for layer in model.layers])]
                              ,'runing_times':[inloop_tm]
                              ,'auc':[roc_auc_score(y_test, preds_mlp[:,1])]
                              ,'f1score':[f1_score(y_test, np.where(preds_mlp[:,1]>PREDS_RATIO,1,0), average='weighted')]
                              ,'precision':[precision_score(y_test, np.where(preds_mlp[:,1]>PREDS_RATIO,1,0), average='weighted')]
                              ,'recall':[recall_score(y_test, np.where(preds_mlp[:,1]>PREDS_RATIO,1,0), average='weighted')]
                              ,'loss':[results[0]]
                              ,'acc':[results[1]]
                              }
              df_result_MLP = df_result_MLP.append(pd.DataFrame(result_dic) ,ignore_index=True)
              print(f"Size of table {df_result_MLP.shape} time: {(time.time() - start_tm)/60:.2f} min")
              #Save Result to google Drive
              df_result_MLP.to_csv(f"/content/drive/MyDrive/Colab Notebooks/BADS7604_DL/data/mlp_result_l{len(model.layers):03d}_c{count_loop:03d}.csv",index=False)
            else :
              print(f"Skip step : {count_loop}")
print(f"Total runing loop : {time.time() - start_tm}")

Skip step : 1
Skip step : 2
Skip step : 3
Skip step : 4
Skip step : 5
Skip step : 6
Skip step : 7
Skip step : 8
Skip step : 9
Size of table (10, 14) time: 0.88 min
Size of table (11, 14) time: 2.33 min
Size of table (12, 14) time: 3.22 min
Size of table (13, 14) time: 4.18 min


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Size of table (14, 14) time: 5.63 min
Size of table (15, 14) time: 6.56 min
Size of table (16, 14) time: 8.00 min
Size of table (17, 14) time: 9.45 min
Size of table (18, 14) time: 10.90 min
Size of table (19, 14) time: 11.78 min


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Size of table (20, 14) time: 13.21 min
Size of table (21, 14) time: 14.09 min
Size of table (22, 14) time: 14.98 min
Size of table (23, 14) time: 16.42 min
Size of table (24, 14) time: 17.87 min
Size of table (25, 14) time: 20.11 min


...

Size of table (164, 14) time: 287.64 min
Size of table (165, 14) time: 287.94 min
Size of table (166, 14) time: 288.25 min
Size of table (167, 14) time: 288.53 min
Size of table (168, 14) time: 288.93 min
Size of table (169, 14) time: 289.59 min
Size of table (170, 14) time: 290.23 min
Size of table (171, 14) time: 290.87 min
Size of table (172, 14) time: 291.63 min
Size of table (173, 14) time: 292.27 min
Size of table (174, 14) time: 292.90 min
Size of table (175, 14) time: 293.64 min
Size of table (176, 14) time: 294.28 min
Size of table (177, 14) time: 295.03 min
Size of table (178, 14) time: 295.79 min
Size of table (179, 14) time: 296.41 min
Size of table (180, 14) time: 297.15 min
Size of table (181, 14) time: 297.79 min
Size of table (182, 14) time: 298.44 min
Size of table (183, 14) time: 299.19 min
Size of table (184, 14) time: 299.95 min
Size of table (185, 14) time: 300.58 min
Size of table (186, 14) time: 301.34 min
Size of table (187, 14) time: 301.96 min


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Size of table (188, 14) time: 302.57 min
Size of table (189, 14) time: 303.19 min
Size of table (190, 14) time: 303.95 min
Size of table (191, 14) time: 304.57 min
Size of table (192, 14) time: 305.17 min
Size of table (193, 14) time: 306.61 min
Size of table (194, 14) time: 308.06 min
Size of table (195, 14) time: 309.50 min
Size of table (196, 14) time: 310.70 min
Size of table (197, 14) time: 312.14 min
Size of table (198, 14) time: 313.36 min
Size of table (199, 14) time: 314.80 min
Size of table (200, 14) time: 315.95 min
Size of table (201, 14) time: 317.11 min
Size of table (202, 14) time: 318.54 min
Size of table (203, 14) time: 319.99 min
Size of table (204, 14) time: 321.12 min
Size of table (205, 14) time: 322.34 min
Size of table (206, 14) time: 323.77 min
Size of table (207, 14) time: 325.21 min
Size of table (208, 14) time: 326.43 min
Size of table (209, 14) time: 327.88 min
Size of table (210, 14) time: 329.08 min
Size of table (211, 14) time: 330.25 min


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Size of table (212, 14) time: 331.38 min
Size of table (213, 14) time: 332.82 min
Size of table (214, 14) time: 333.98 min
Size of table (215, 14) time: 335.41 min
Size of table (216, 14) time: 336.55 min
Total runing loop : 20192.990369796753


In [ ]:
Save result : https://docs.google.com/spreadsheets/d/1CtRk7Nco5w2Nefw355E39Xh0Xkl71nQDtBV6u_2n-5U/edit#gid=1840033743